# IAM COMPACT - Study 8 - Global Trade Dynamics
<br>

### Supply-Use Input-Output model for impact assessment of different import patterns on key European energy commodities (i.e. BEVs)  MARIO model based on EXIOBASE HYBRID v3.3.18
<br>
The model proposed in this repository is built using the MARIO framework, starting from the Exiobase Hybrid v3.3.18 multi-regional input-output database, properly adjusted and extended for the purposes of this study.  

<br>
Main references for software and tools and data sources:

- Tahavori et al., 2023, ‘MARIO: A Versatile and User-Friendly Software for Building Input-Output Models’, <i>Journal of Open Research Software</i>, 11(1 ), p. 14. Available at: https://doi.org/10.5334/jors.473.
- Merciai & Schmidt, 2021, 'EXIOBASE HYBRID v3 - 2011 (3.3.18)', <i>Zenodo</i>. Available at: https://doi.org/10.5281/zenodo.10148587
- EMBER, 2024, 'Yearly Electricity Data'. Available at: https://ember-energy.org/data/yearly-electricity-data/ 

<br>
<br>
<br>


### Outline
<br>
This notebook allows to replicate the modelling exercise starting from the parsing of the raw database, the implmentation of updated electricity mixes as well as the fixing of some out-of-scale value added coefficients in some key sectors for the analysis.
<br>

- [Software installation](#software-installation)
- [Database download](#database-download)
- [Baseline building](#baseline-building)
    - [Setting paths YAML](#setting-paths-yaml)
    - [Parsing and aggregating raw database](#parsing-and-aggregating-raw-database)
    - [Updating raw database electricity mixes and value added coefficients](#updating-raw-database-electricity-mixes-and-value-added-coefficients)
    - [Adding new supply chains](#adding-new-supply-chains)
    - [Exporting baseline database](#exporting-baseline-database)

<br>
<br>
<br>

---

<br>


### Software installation

<br>
(<i>last update 13th November 2024</i>),
<br>
<br>

This model runs on Python 3.10 and adopts a specific branch from MARIO GitHub repository, not yet available from installation via pip. Therefore, you need to:
<br>
<br>

1. Clone the repository in a given directory: 
   
   git clone https://github.com/it-is-me-mario/MARIO.git <target_folder>
   
   <br>

2. Ensure to locate your terminal in the correct directory path

   cd <target_folder>
   
   <br>
   
3. Checkout to the correct branch, named 'dev_addsector_improved'

   git checkout dev_addsector_improved

    <br>

4. Once you installed the correct version of MARIO, create a new python environment.

   conda create -n NAME_OF_ENVIRONMENT python=3.10

   <br>
   
5. Install the local version of MARIO by running the following commands 

   python setup.py sdist bdist_wheel

   python -m pip install -e . 

   <br>

6. To run this model, other two packages are needed: pint and country_converter. They can be installed via pip:

   pip install pint

   pip install country_converter

   <br>

7. Due to some recent issues in pint, you need also a specific version of the flexparser package

   pip install flexparser==0.3.1

   <br>

You should be all set at this point

<br>
<br>
<br>

---

<br>


### Database download 

The raw database used for this model is Exiobase Hybrid v3.3.18.  
Download the whole folder from the [Zenodo repository](https://zenodo.org/records/10148587).

Save it to a desired directory.

<br>
<br>
<br>

---

<br>


### Baseline building 

To build the baseline of this model, several steps are required. 

#### Setting paths YAML
First of all, you need to properly set the paths YAML file, specifically:
- <i>database/exiobase</i>: provide here the folder where you stored the raw Exiobase database downloaded from Zenodo
- <i>database/baseline</i>: provide here the folder where you intend to store the baseline database once it's ready

<br> 
Continue by importing dependencies and loading the YAML file.

In [ ]:
import mario
import yaml
import os

with open('paths.yml', 'r') as file:
    paths = yaml.safe_load(file)


#### Parsing and aggregating raw database
Parse the raw database with MARIO

In [ ]:
db = mario.parse_from_txt(
    path = os.path.join(paths['database']['exiobase'],'flows'),     # specify the inner "flows" folder
    mode = 'flows',                                                 # specify the mode (flows or coefficients)
    table = 'SUT',                                                  # the table is in SUT format
    )

A first level of aggregation is needed on the electricity commodities and activities, in order to update the electricity mixes from 2011 to 2023 (supported by latest data available on the EMBER database). Updating the electricity mixes helps in providing a more realistic representation of the current state of the energy system, which is also the major global source of GHG emissions.

<br>
A MARIO template is already provided in this repository and we use MARIO 'aggregate' method to perform the aggregation

In [ ]:
import warnings
warnings.filterwarnings("ignore")   # ignore warnings from other libraries (i.e. pandas) which may be misleading for the scope of this model

db.aggregate(
    io = paths['mario_templates']['aggregations']['raw_to_baseline'],
    ignore_nan=True, 
    )

We need to import the updated electricity mixes from EMBER data, and map such data with the regional granularity of EXIOBASE. A dedicated function is available in the <i>supporting_functions/ember_remapping.map_ember_to_classification</i> script, which exploits the <i>country_converter</i> Python library and the EMBER data that are already provided in this repository 

In [ ]:
from supporting_functions.ember_remapping import map_ember_to_classification

ee_mix = map_ember_to_classification(
    path = paths['data']['ember'],
    classification = 'EXIO3',
    year = 2023,
    mode = 'mix',
)

#### Updating raw database electricity mixes and value added coefficients
The electricity mix update can be done by using the traditional shock function in MARIO, but we prepared a more efficient one tailored for this application. 

In [ ]:
from supporting_functions.raw_database_update import electricity_mixes

db = electricity_mixes(db, ee_mix)

Other specific coefficients of some key industrial activities where considered out of scale and where fixed in this application. Check the <i>supporting_functions/raw_database_update.value_added_coefficients</i> function for more details 

In [ ]:
from supporting_functions.raw_database_update import value_added_coefficients

db = value_added_coefficients(db)

#### Adding new supply chains

At this stage, we need to add the new supply chains, characterized in the <i>inputs/inventories/master.xlsx</i> file. 


The new activities represent the manufacturing of LFP, NMC, NCA batteries and electric vehicles at global level and are added as an extension to the raw Exiobase database in a similar way as was done by [Rinaldi et al. (2024)](https://iopscience.iop.org/article/10.1088/1748-9326/ad5bf1) (hybrid LCA approach).

In [ ]:
db.read_add_sectors_excel(paths['data']['inventories'],read_inventories=True)  # read inventories from excel file
db.add_sectors()  # add sectors to the database

While the <i>inputs/inventories/master.xlsx</i> file introduces the new activities in all countries, it assumes each country manufactures the new commodities fully domestically. 

<br>
The import shares of given commodities (i.e. batteries) need to be updated. The data used are available in <i>inputs/scenarios/import_shares.xlsx</i>. Here we provided import shares for the baseline but also for other scenarios implemented later.

<br>

Again, the shock function of MARIO was bypassed by a tailored function for this application

In [ ]:
from supporting_functions.raw_database_update import get_import_shares
from supporting_functions.raw_database_update import import_coefficients

import_shares, regions_clusters, commodities_clusters = get_import_shares(paths['data']['import_shares'])
db = import_coefficients(db, import_shares, regions_clusters, commodities_clusters)

#### Exporting baseline database
The baseline database is now ready and can be exported in the desired directory in txt format

In [ ]:
db.to_txt(
    paths['database']['baseline'],
    flows=False,
    coefficients=True
    )

Baseline database is now ready and stored in the desired directory

<br>
<br>
<br>

---

<br>